ターミナルに  
export HEADQUARTER_DATABASE_ID  
と  
export BOOKMARK_DATABASE_ID  
と  
NOTION_INTEGRATION_TOKEN  
をしてから実行してください。

In [99]:
import os
os.environ['OPENAI_API_KEY'] = os.environ.get('OPENAI_API_KEY')
os.environ['NOTION_INTEGRATION_TOKEN'] = os.environ.get('NOTION_INTEGRATION_TOKEN')
os.environ["GEMINI_API_KEY"] = os.environ.get("GEMINI_API_KEY")
# os.environ['HEADQUARTER_DATABASE_ID'] = os.environ.get('HEADQUARTER_DATABASE_ID')
# os.environ['BOOKMARK_DATABASE_ID'] = os.environ.get('BOOKMARK_DATABASE_ID')
# headquarter_database_id = os.environ.get("HEADQUARTER_DATABASE_ID")
# BOOKMARK_DATABASE_ID = os.environ.get("BOOKMARK_DATABASE_ID")
# integration_token = os.environ.get("NOTION_INTEGRATION_TOKEN")
# print(integration_token)
# print(headquarter_database_id)
# print(BOOKMARK_DATABASE_ID)
integration_token = 'secret_NqX3JTPq6E3Zj5cnYanrfZfQ6GVqmNxGyjsVFe3feGM'
HEADQUARTER_DATABASE_ID = '7d6f473167454ca3936fece11a0b692a'
BOOKMARK_DATABASE_ID = 'd979925d021f499d90a5ab9cfede7487'


# NotionをGraphRAGで扱うための準備

In [100]:
import logging
import sys
from llama_index.core import SummaryIndex
from IPython.display import Markdown, display
import os
from tqdm import tqdm
import requests
from llama_index.readers.notion import NotionPageReader
import urllib3
# ログレベルの設定
# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG, force=True)
urllib3.disable_warnings()
logging.getLogger("urllib3").setLevel(logging.WARNING)

In [101]:
from llama_index.core import Settings, VectorStoreIndex
from llama_index.llms.openai import OpenAI
# from llama_index.llms.gemini import Gemini # ColabじゃないとGCPの設定が必要？
# from llama_index.embeddings.gemini import GeminiEmbedding # ColabじゃないとGCPの設定が必要？

# データ準備

In [102]:
# notion_test.py
def get_database_pages(databese_id):
    url = f"https://api.notion.com/v1/databases/{databese_id}/query"
    headers = {
        "Notion-Version": "2022-06-28",
        'Authorization': 'Bearer ' + integration_token,
        'Content-Type': 'application/json',
    }
    response = requests.post(url, headers=headers)
    return response.json()

def extract_page_titles(database_data):
    page_titles = []
    for page in database_data.get('results', []):
        properties = page.get('properties', {})
        for prop in properties.values():
            if prop['type'] == 'title':
                title = prop['title'][0]['plain_text'] if prop['title'] else ''
                page_titles.append(title)
                break
    return page_titles


# # 使用例
# database_data = get_database_pages(HEADQUARTER_DATABASE_ID)
# page_titles = extract_page_titles(database_data)
# print(page_titles)

def get_page_title(page_id):
    url = f"https://api.notion.com/v1/pages/{page_id}"
    headers = {
        "Notion-Version": "2022-06-28",
        'Authorization': 'Bearer ' + NOTION_API_KEY,
        'Content-Type': 'application/json',
    }
    response = requests.get(url, headers=headers)
    page_data = response.json()
    
    # ページのプロパティからタイトルを抽出
    properties = page_data.get('properties', {})
    for prop in properties.values():
        if prop['type'] == 'title':
            return prop['title'][0]['plain_text'] if prop['title'] else ''
    
    # タイトルが見つからない場合は空文字列を返す
    return ''

# 使用例
# page_id = "page_id here"
# title = get_page_title(page_id)
# print(f"Page Title: {title}")

In [104]:
# Initialize NotionPageReader
reader = NotionPageReader(integration_token=integration_token)

# Load data from Notion
# 指定したデータベースが持つページのデータを読み込む
headquarter_documents = reader.load_data(page_ids=reader.query_database(HEADQUARTER_DATABASE_ID))  # List of page IDs to load
bookmark_documents = reader.load_data(page_ids=reader.query_database(BOOKMARK_DATABASE_ID))  # List of page IDs to load

# 辞書型に整形
# ページタイトルをキー、ページ内容をバリューとする辞書を作成
docs = {}
for doc in tqdm(bookmark_documents):
    page_title, page_contents = get_page_title(doc.id_), doc.text
    docs[page_title] = page_contents

# 辞書をtxtに書き出す。キーをファイル名、値をコンテンツとして書き出す
for key, value in tqdm(docs.items()):
    file_name = key.replace('.', '_').replace('/', '-')
    with open(f"ragtarget/input/{file_name}.txt", "w") as f:
        f.write(value)

100%|██████████| 39/39 [00:00<00:00, 2183.63it/s]


# RAGtest
bookmark_documentsについて質問

In [105]:
documents = bookmark_documents

index = SummaryIndex.from_documents(documents)
# set Logging to DEBUG for more detailed outputs
query_engine = index.as_query_engine()
response = query_engine.query("SQLを勉強するのに良い資料は？")
display(Markdown(f"{response}"))

DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: The National Payments Corporation of India, the...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: a. 10時間コース
10時間コースでは，深層学習の背景にある数学や理論を飛ばし，短時間でプロ...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: (TensorFlow)
Deep Learning実践開発講座（DL4US）
 

	Ker...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: (
TensorFlow
)のコースを中心に紹介しましたが，200時間コースでは
PyTorc...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 数学の勉強（学習想定時間：40時間）
機械学習や深層学習の理解に必要な「線形代数」「微分積分」...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: プログラミングの勉強（学習想定時間：40時間）
機械学習や深層学習を理解し実装するために，プロ...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: PythonやNumPyを用いたデータ分析の方法を学ぶ（学習想定時間：30時間）
3. プログ...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Introduction to Deep 

DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 勉強について
エンジニアの皆さん。エンジニア以外の皆さん。

 ・勉強しようと思っているけど、...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: SEGA
SEGAはゲーム会社なので、数学についての資料を展開していますね。
クォータニオンと...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 昨年、
株式会社松尾研究所
（東京大学松尾研とビジョンを共有）に転職しました。
現在は技術顧問...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: ai」をメインにしています。
1.2 最新の技術情報
技術系の情報については、はてなブックマー...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: URLべた書きを再掲いたします。
https://papers.labml.ai/papers...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: （上記3つのメルマガの雰囲気）
以上、毎朝チェックする情報の紹介でした。
続いては、週単位でチ...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: 今回は、今話題の ChatGPT と Notion の連携についてのご紹介です。 AI機能は ...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Notion の新規ページにテーブルを作成する
](https://notion-lab.jp...
DEBUG:llama_index.core.node_parse

SQLを学ぶための良い資料として、GMOペパボのSQL研修資料が基本から始まり、RDBMSの歴史やデータベース設計に触れているので初心者にも理解しやすいです。また、Business Insiderやダイヤモンド・シグナルもSQLを含むテクノロジー系の情報を提供しているので、参考になるでしょう。